In [6]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt

import json

tokenizer = Tokenizer()

# wordInext 목록 받아오기
with open('LSTM_module_ver2.0_wordIndex.json') as f:
    word_index = json.load(f)
    tokenizer.word_index = word_index
    
with open('LSTM_module_ver2.0_variable.json') as f:
    var = json.load(f)
    
max_data = var['max_data'] # 데이터 문자열 최대 길이
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)

model = tf.keras.models.load_model('LSTM_module_ver2.0.h5')
model.summary()

okt = Okt()

def preTreatment(data):
    stopwords = ['\n','.','?',',','']
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    print(data)
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 100로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    if (score > 0.5):
        print(' {:.2f}% 확률로 스팸입니다.'.format(score * 100))
    else:
        print(' {:.2f}% 확률로 스팸이 아닙니다.'.format((1 - score) * 100))
    
    print('-------------------------------------------------------------------------')
        

# 음성파일 STT 변환
import speech_recognition as sr

r = sr.Recognizer()
harvard = sr.AudioFile('spam1.wav') # 100MB 용량 제한
with harvard as source:
    audio = r.record(source)

t = r.recognize_google(audio, language='ko-KR')

temp_X = []
x = [t[i:i+max_data] for i in range(0, len(t), max_data)] # 텍스트 파일 중 150 길이로 데이터 길이 제한

for s in x:
    if len(s) > min_data: # 길이가 50 이하인 데이터 삭제
        temp_X.append(s)
x = temp_X
for txt in x:
    print(txt)
    data = preTreatment(txt)
    sentiment_predict(data)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2466000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,583,377
Trainable params: 2,583,377
Non-trainable params: 0
_________________________________________________________________
여보세요 여보세요네네 여기 서울중앙지검 잠시 통화 괜찮으십니까 다름이 아니라 본인 명의도용사건 관련해서 몇 가지 사실 확인 차 연락 드렸어요 저는 서울중앙지검 특별수사팀 제일 보강은 영숙아이라고 합니다 예 부모님께서 대구 달서구 출신인 41 - 7 감동이라는 사람 아는
[[1013, 1013, 23, 23, 316, 77, 229, 810, 83, 805, 828, 530, 190, 452, 11, 73, 389, 892, 215, 752, 1, 223, 390, 378, 262, 331, 159, 177, 72, 15, 77, 229, 810, 83, 805, 5210, 727, 793, 596, 11507, 22, 460, 1, 142, 927, 269, 170